In [0]:
import pandas as pd
import numpy as np

In [0]:
data=pd.read_csv('cus_id')

In [0]:
data

,Unnamed: 0,CustomerID,Recency,frequency,Amount,recency_log,frequency_log,amount_log
0,0,12346.0,326.0,1,77183.60,5.786897,0.000000,11.253942
1,1,12347.0,2.0,7,4310.00,0.693147,1.945910,8.368693
2,2,12348.0,75.0,4,1797.24,4.317488,1.386294,7.494007
3,3,12349.0,19.0,1,1757.55,2.944439,0.000000,7.471676
4,4,12350.0,310.0,1,334.40,5.736572,0.000000,5.812338
...,...,...,...,...,...,...,...,...
4333,4333,18280.0,278.0,1,180.60,5.627621,0.000000,5.196285
4334,4334,18281.0,181.0,1,80.82,5.198497,0.000000,4.392224
4335,4335,18282.0,8.0,2,178.05,2.079442,0.693147,5.182064
4336,4336,18283.0,4.0,16,2094.88,1.386294,2.772589,7.647252


In [0]:
work=data.drop(['Unnamed: 0','recency_log','frequency_log','amount_log'],axis=1)

In [0]:
work

,CustomerID,Recency,frequency,Amount
0,12346.0,326.0,1,77183.60
1,12347.0,2.0,7,4310.00
2,12348.0,75.0,4,1797.24
3,12349.0,19.0,1,1757.55
4,12350.0,310.0,1,334.40
...,...,...,...,...
4333,18280.0,278.0,1,180.60
4334,18281.0,181.0,1,80.82
4335,18282.0,8.0,2,178.05
4336,18283.0,4.0,16,2094.88


In [0]:
work=work.drop(['CustomerID'],axis=1)

In [0]:
quartiles = work.quantile(q=[0.25,0.50,0.75])


In [0]:
quartiles=quartiles.to_dict()

In [0]:
quartiles

{'Amount': {0.25: 307.41499999999996,
  0.5: 674.4849999999999,
  0.75: 1661.7400000000007},
 'Recency': {0.25: 18.0, 0.5: 51.0, 0.75: 142.0},
 'frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 5.0}}

In [0]:
def R_score(val,p='Recency'):
    if val <= quartiles[p][0.25]:
        return 1
    elif val <= quartiles[p][0.50]:
        return 2
    elif val <= quartiles[p][0.75]: 
        return 3
    else:
        return 4
    
def F_score(val,p='frequency'):
    if val <= quartiles[p][0.25]:
        return 4
    elif val <= quartiles[p][0.50]:
        return 3
    elif val <= quartiles[p][0.75]: 
        return 2
    else:
        return 1


def M_score(val,p='Amount'):
    if val <= quartiles[p][0.25]:
        return 4
    elif val <= quartiles[p][0.50]:
        return 3
    elif val <= quartiles[p][0.75]: 
        return 2
    else:
        return 1

In [0]:
scored=work
scored['R_score']=scored['Recency'].apply(R_score)
scored['F_score']=scored['frequency'].apply(F_score)
scored['M_score']=scored['Amount'].apply(M_score)

In [0]:
scored

,Recency,frequency,Amount,R_score,F_score,M_score
0,326.0,1,77183.60,4,4,1
1,2.0,7,4310.00,1,1,1
2,75.0,4,1797.24,3,2,1
3,19.0,1,1757.55,2,4,1
4,310.0,1,334.40,4,4,3
...,...,...,...,...,...,...
4333,278.0,1,180.60,4,4,4
4334,181.0,1,80.82,4,4,4
4335,8.0,2,178.05,1,3,4
4336,4.0,16,2094.88,1,1,1


In [0]:
scored['RFM_score']=scored.R_score.map(str)+scored.F_score.map(str)+scored.M_score.map(str)

In [0]:
scored=scored.join(pd.DataFrame(data['CustomerID']))

In [0]:
final=scored.sort_values(by=['RFM_score','Amount'],ascending=[True, False])

In [0]:
final

,Recency,frequency,Amount,R_score,F_score,M_score,RFM_score,CustomerID
1689,2.0,73,280206.02,1,1,1,111,14646.0
4201,1.0,60,259657.30,1,1,1,111,18102.0
3728,8.0,46,194550.79,1,1,1,111,17450.0
1879,1.0,201,143825.06,1,1,1,111,14911.0
1333,10.0,55,117379.63,1,1,1,111,14156.0
...,...,...,...,...,...,...,...,...
3486,262.0,1,25.50,4,4,4,444,17102.0
2558,372.0,1,15.00,4,4,4,444,15823.0
3960,263.0,1,15.00,4,4,4,444,17763.0
4098,249.0,1,12.75,4,4,4,444,17956.0


In [0]:
print("Top 5 customers are:")
final.head()['CustomerID']

Top 5 customers are:


1689    14646.0
4201    18102.0
3728    17450.0
1879    14911.0
1333    14156.0
Name: CustomerID, dtype: float64

In [0]:
#Customers That spend lot of money but they don't shop frequently
filterr=final.loc[(final['M_score']==4) & (final['F_score']==1)]

In [0]:
print("Customers That spend lot of money but they don't shop frequently")
filterr['CustomerID']

Customers That spend lot of money but they don't shop frequently


3164    16668.0
4124    17988.0
4155    18037.0
Name: CustomerID, dtype: float64

**Cluster based RFM score analysis**

In [0]:
asign=pd.read_csv('/content/assinged.csv')

In [0]:
asign

,Column 0,CustomerID,amount_log,recency_log,frequency_log,Assignments,DistancesToClusterCenter no.0,DistancesToClusterCenter no.1,DistancesToClusterCenter no.2,DistancesToClusterCenter no.3,DistancesToClusterCenter no.4,DistancesToClusterCenter no.5,DistancesToClusterCenter no.6
0,0,12346,3.698719,1.409894,-1.048610,1,3.877149,3.858889,4.802091,4.875674,4.813504,4.753682,3.871522
1,1,12347,1.410201,-2.146498,1.111836,5,3.614663,2.497526,3.411605,1.356515,4.614537,1.333829,1.720836
2,2,12348,0.716419,0.383971,0.490522,1,1.242979,0.497032,2.070716,3.053071,2.455702,2.017343,1.227141
3,3,12349,0.698706,-0.574674,-1.048610,2,1.706740,1.579999,1.520879,3.745962,2.461262,1.884846,2.361659
4,4,12350,-0.617446,1.374758,-1.048610,4,1.040869,2.219611,1.722261,5.249346,0.562042,3.492694,3.453037
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333,4333,18280,-1.106086,1.298690,-1.048610,4,1.332027,2.440629,1.673667,5.467014,0.206501,3.598092,3.697011
4334,4334,18281,-1.743850,0.999081,-1.048610,4,1.813741,2.757437,1.659570,5.703498,0.649581,3.684474,4.007735
4335,4335,18282,-1.117365,-1.178605,-0.279044,2,2.235813,2.041251,1.047668,4.064853,2.387726,1.721129,2.875074
4336,4336,18283,0.837969,-1.662552,2.029655,3,3.559090,2.392175,3.495654,1.192172,4.532437,1.608189,1.488083


In [0]:
df=scored.join(pd.DataFrame(asign['Assignments']))

In [0]:
df

,Recency,frequency,Amount,R_score,F_score,M_score,RFM_score,CustomerID,Assignments
0,326.0,1,77183.60,4,4,1,441,12346.0,1
1,2.0,7,4310.00,1,1,1,111,12347.0,5
2,75.0,4,1797.24,3,2,1,321,12348.0,1
3,19.0,1,1757.55,2,4,1,241,12349.0,2
4,310.0,1,334.40,4,4,3,443,12350.0,4
...,...,...,...,...,...,...,...,...,...
4333,278.0,1,180.60,4,4,4,444,18280.0,4
4334,181.0,1,80.82,4,4,4,444,18281.0,4
4335,8.0,2,178.05,1,3,4,134,18282.0,2
4336,4.0,16,2094.88,1,1,1,111,18283.0,3


In [0]:
#Now we will find out the customers which are most frequent in each cluster
def most_freq(clust):
  lst=df.loc[(df['Assignments']==clust) & (df['F_score']==1)]['CustomerID'].values.tolist()
  return lst
def most_freq_alt(clust):
  lst=df.loc[(df['Assignments']==clust) & (df['F_score']<=2)]['CustomerID'].values.tolist()
  return lst
def most_rec(clust):
  lst=df.loc[(df['Assignments']==clust) & (df['R_score']<=2)]['CustomerID'].values.tolist()
  return lst

In [0]:
most_0=most_freq(0)+most_rec(0) #was Getting only 1 customer using most_freq

In [0]:
most_0

[15107.0,
 12398.0,
 12556.0,
 12596.0,
 13601.0,
 14754.0,
 15068.0,
 15468.0,
 15469.0,
 15821.0,
 16208.0,
 16260.0,
 16593.0,
 16751.0,
 17354.0,
 17423.0,
 17459.0,
 17764.0,
 18252.0]

In [0]:
most_1=most_freq(1)

In [0]:
most_1

[12455.0,
 12599.0,
 12643.0,
 12676.0,
 12691.0,
 12778.0,
 12853.0,
 12868.0,
 12947.0,
 13138.0,
 13208.0,
 13317.0,
 13477.0,
 13538.0,
 13549.0,
 13717.0,
 13728.0,
 13752.0,
 14078.0,
 14201.0,
 14210.0,
 14312.0,
 14344.0,
 14404.0,
 14407.0,
 14409.0,
 14410.0,
 14414.0,
 14434.0,
 14472.0,
 14487.0,
 14507.0,
 14541.0,
 14573.0,
 14738.0,
 14883.0,
 14898.0,
 14948.0,
 14976.0,
 15038.0,
 15220.0,
 15260.0,
 15304.0,
 15508.0,
 15860.0,
 15996.0,
 16027.0,
 16059.0,
 16098.0,
 16170.0,
 16184.0,
 16266.0,
 16306.0,
 16419.0,
 16455.0,
 16681.0,
 16725.0,
 16918.0,
 16996.0,
 17007.0,
 17027.0,
 17109.0,
 17214.0,
 17227.0,
 17284.0,
 17287.0,
 17460.0,
 17612.0,
 17723.0,
 17848.0,
 17867.0,
 17874.0,
 17888.0,
 17894.0,
 18116.0,
 18168.0,
 18177.0,
 18236.0,
 18260.0]

In [0]:
most_2=most_freq(2)

In [0]:
most_2

[]

In [0]:
most_2=most_freq_alt(2)

In [0]:
most_2

[12498.0,
 13058.0,
 13277.0,
 13502.0,
 13606.0,
 13666.0,
 14287.0,
 14385.0,
 14765.0,
 14798.0,
 14836.0,
 15516.0,
 15781.0,
 15901.0,
 16330.0,
 16462.0,
 16478.0,
 16930.0,
 17024.0,
 17371.0,
 17573.0,
 17816.0,
 17892.0,
 17917.0,
 18059.0,
 18081.0,
 18127.0]

In [0]:
most_3=most_freq(3)

In [0]:
most_3

[12362.0,
 12415.0,
 12417.0,
 12433.0,
 12437.0,
 12471.0,
 12474.0,
 12490.0,
 12540.0,
 12569.0,
 12583.0,
 12621.0,
 12662.0,
 12681.0,
 12682.0,
 12683.0,
 12709.0,
 12720.0,
 12747.0,
 12748.0,
 12766.0,
 12839.0,
 12841.0,
 12901.0,
 12921.0,
 12931.0,
 12955.0,
 12971.0,
 12989.0,
 13001.0,
 13004.0,
 13013.0,
 13018.0,
 13069.0,
 13078.0,
 13081.0,
 13089.0,
 13090.0,
 13098.0,
 13102.0,
 13113.0,
 13199.0,
 13225.0,
 13230.0,
 13263.0,
 13267.0,
 13269.0,
 13319.0,
 13408.0,
 13418.0,
 13426.0,
 13458.0,
 13468.0,
 13534.0,
 13576.0,
 13599.0,
 13668.0,
 13694.0,
 13709.0,
 13756.0,
 13767.0,
 13769.0,
 13777.0,
 13798.0,
 13854.0,
 13881.0,
 13969.0,
 13985.0,
 14031.0,
 14032.0,
 14051.0,
 14056.0,
 14060.0,
 14062.0,
 14085.0,
 14088.0,
 14092.0,
 14096.0,
 14110.0,
 14132.0,
 14135.0,
 14156.0,
 14180.0,
 14189.0,
 14191.0,
 14194.0,
 14217.0,
 14258.0,
 14282.0,
 14291.0,
 14298.0,
 14367.0,
 14395.0,
 14397.0,
 14401.0,
 14415.0,
 14527.0,
 14534.0,
 14547.0,
 14560.0,


In [0]:
len(most_3)

228

In [0]:
most_4=most_freq(4)

In [0]:
most_4[:5]

[18037.0, 13003.0, 18037.0]

In [0]:
len(most_4)

1

In [0]:
most_4=most_freq(4)+most_freq_alt(4)

In [0]:
len(most_4)

3

In [0]:
most_5=most_freq(5)

In [0]:
most_5[:5]

[12347.0, 12423.0, 12464.0, 12528.0, 12584.0]

In [0]:
len(most_5)

114

In [0]:
most_6=most_freq(6)

In [0]:
most_6[:5]

[12352.0, 12388.0, 12395.0, 12428.0, 12431.0]

In [0]:
len(most_6)

449